Module System
---

Rust has a number of features that allow you to manage your code’s organization,  
including which details are exposed, which details are private, and what names  
are in each scope in your programs. These features, sometimes collectively  
referred to as the module system, include:  

- **`Packages`**: A Cargo feature that lets you build, test, and share crates
- **`Crates`**: A tree of modules that produces a library or executable
- **`Modules and use`**: Let you control the organization, scope, and privacy of paths
- **`Paths`**: A way of naming an item, such as a struct, function, or module

> In this chapter, we’ll cover all these features, discuss how they interact,  
and explain how to use them to manage scope. By the end, you should have a solid  
understanding of the module system and be able to work with scopes like a pro!

Packages and Crates
---

### Crate
A crate is the smallest amount of code that the Rust compiler considers at a  
time. Even if you run rustc rather than cargo and pass a single source code file  
(as we did all the way back in the “Writing and Running a Rust Program” section  
of Chapter 1), the compiler considers that file to be a crate. Crates can  
contain modules, and the modules may be defined in other files that get compiled  
with the crate, as we’ll see in the coming sections.

A crate can come in one of two forms:  

- **`Binary crates`** are programs you can compile to an executable that you can run,  
such as a command-line program or a server. Each must have a function called  
main that defines what happens when the executable runs. All the crates we’ve  
created so far have been binary crates.
- **`Library crates`** don’t have a main function, and they don’t compile to an  
executable. Instead, they define functionality intended to be shared with  
multiple projects. For example, the rand crate we used in Chapter 2 provides  
functionality that generates random numbers. Most of the time when Rustaceans  
say “crate”, they mean library crate, and they use “crate” interchangeably with  
the general programming concept of a “library".

The crate root is a source file that the Rust compiler starts from and makes up  
the root module of your crate (we’ll explain modules in depth in the  
“Defining Modules to Control Scope and Privacy” section).

### Package

A package is a bundle of one or more crates that provides a set of functionality.  
A package contains a Cargo.toml file that describes how to build those crates.  
Cargo is actually a package that contains the binary crate for the command-line  
tool you’ve been using to build your code. The Cargo package also contains a  
library crate that the binary crate depends on. Other projects can depend on the  
Cargo library crate to use the same logic the Cargo command-line tool uses.

A package can contain : 
- as many binary crates as you like 
- but at most only one library crate. 
- a package must contain at least one crate, whether that’s a library or  
binary crate.

### Cargo

Let’s walk through what happens when we create a package.  First, we enter the  
command cargo new :

```sh
$ cargo new my-project
     Created binary (application) `my-project` package
$ ls my-project
Cargo.toml
src
$ ls my-project/src
main.rs
```

After we run cargo new, we use ls to see what Cargo creates.  

In the project directory, there’s a : 
- **`Cargo.toml`** file, giving us a **`package`** 
- a **`src`** directory that contains **`main.rs`** 

Open Cargo.toml in your text editor, and note there’s no mention of src/main.rs.  

That's because Cargo follows an IMPLICIT convention that : 
- **`src/main.rs`** is the crate **`root of a binary crate`** with the same name  
as the package. 
- if the package directory contains **`src/lib.rs`**, the package contains a  
library crate with the same name as the package, and src/lib.rs is its crate root. 

Cargo passes the crate root files to **`rustc`** to build the library or binary.  

Here, we have a package that only contains src/main.rs, meaning:  
- it only contains a binary crate named my-project 

If a package contains src/main.rs and src/lib.rs :
- it has two crates: a binary and a library,  
- both with the same name as the package.  

A package can have multiple binary crates by placing files in the src/bin  
directory: each file will be a separate binary crate.

Defining Modules to Control Scope and Privacy
---

In this section, we’ll talk about modules and other parts of the module system :
- **`paths`** that allow you to name items 
- the **`use`** keyword that brings a path into scope 
- the **`pub`** keyword to make items public.  

We’ll also discuss the :

- **`as`** keyword, 
- **`external packages`**  
- and the **`glob`** operator.

First, we’re going to start with a list of rules for easy reference when you’re  
organizing your code in the future. Then we’ll explain each of the rules in  
detail.

### Module Cheat Sheet

Here we provide a quick reference on how :  

- **`modules`**, 
- **`paths`**, 
- the **`use`** keyword, and  
- the **`pub`** keyword  

work in the compiler, and how most developers organize their code.  We’ll be  
going through examples of each of these rules throughout this chapter, but this  
is a great place to refer to as a reminder of how modules work.  

- **`Start from the crate root:`** When compiling a crate, the compiler first  
looks in the crate root file :  
    - usually src/lib.rs for a **`library crate`** or  
    - src/main.rs for a **`binary crate`**  
    - for **`code to compile`**.
- **`Declaring modules:`** In the crate root file, you can declare new modules;  
say, you declare a “garden” module with mod garden;. The compiler will look for  
the module’s code in these places:
    - Inline, within curly brackets that replace the semicolon following mod  
    garden
    - In the file src/garden.rs  
    - In the file src/garden/mod.rs  
- **`Declaring submodules:`** In any file other than the crate root, you can  
declare submodules. For example, you might declare `mod vegetables;` in  
src/garden.rs. The compiler will look for the submodule’s code within the  
directory named for the parent module in these places:  
    - Inline, directly following mod vegetables, within curly brackets instead  
    of the semicolon
    - In the file src/garden/vegetables.rs  
    - In the file src/garden/vegetables/mod.rs  
- **`Paths to code in modules:`** Once a module is part of your crate, you can  
refer to code in that module from anywhere else in that same crate, as long as  
the privacy rules allow, using the path to the code. For example, an `Asparagus`  
type in the garden vegetables module would be found at  
`crate::garden::vegetables::Asparagus`.  
- **`Private vs public:`** Code within a module is private from its parent  
modules by default. To make a module public, declare it :  
    - with `pub mod` instead of `mod.`  
To make items within a public module public as well : 
    - use `pub` before their declarations.
- **`The use keyword:`** Within a scope, the `use` keyword creates shortcuts to  
items to reduce repetition of long paths. In any scope that can refer to :  
    - `crate::garden::vegetables::Asparagus`,  
    - you can create a shortcut with `use crate::garden::vegetables::Asparagus;` 
    - and from then on you only need to write `Asparagus` to make use of that  
    type in the scope.

Here we create a binary crate named backyard that illustrates these rules.  
The crate’s directory, also named backyard, contains these files and  
directories:  

```sh
backyard  
├── Cargo.lock  
├── Cargo.toml  
└── src  
    ├── garden  
    │   └── vegetables.rs  
    ├── garden.rs  
    └── main.rs  
```

The crate root file in this case is `src/main.rs`, and it contains:

Filename: `src/main.rs`

I apologize for the confusion in my previous response. It seems that the `:code` command I suggested is not supported in the Rust kernel for Jupyter Notebook. 

In this case, you can use a workaround by defining the module directly in the notebook cell. Copy the content of your `vegetable.rs` file into a new cell, and declare the module using the `mod` keyword:

```rust
mod vegetable {
    #[derive(Debug)]
    pub struct Asparagus {}
}
```

Now you should be able to use the `vegetable` module in your Jupyter Notebook:

```rust
let asparagus = vegetable::Asparagus {};
println!("Asparagus: {:?}", asparagus);
```

Keep in mind that this approach may not be ideal for larger modules or more complex project structures. The Rust kernel for Jupyter Notebook is still a relatively new and evolving ecosystem, and some advanced features may not be available or work differently compared to a standard Rust development environment.

In [13]:
// Because we are in a Jupyter cell environment, we are inlining this module
// Pretend that it's in ../src/garden/vegetable.rs lol
mod vegetable {
    #[derive(Debug)]
    pub struct Asparagus {}

    #[derive(Debug)]
    pub struct Carrot {}

    #[derive(Debug)]
    pub struct Beet {}
}

{   // closure starts
    // pretend that we are importing this crate from ../src/garden/vegetable.rs
    use crate::vegetable;

    let plant = vegetable::Asparagus{}; // plant(a) comes into scope
    println!("I am growing {:?}", plant);
    // @todo : why can't I mut plant instead of shadowing it?
    // ANSWER:
    // Shadowing is the correct approach where the variable's TYPE is CHANGING
    // In this case, you are assigning completely different types to the `plant` 
    // variable: 
    // - first an instance of `Asparagus`, 
    // - and then an instance of `Beet`. 
    let plant = vegetable::Beet{}; // plant(b) comees into scope and shadows plant(a)
    println!("But Imma only going to eat {:?}", plant);
}   // closure ends
    // plant(b) goes out of scope
    // plant(a) goes out of scope

I am growing Asparagus
But Imma only going to eat Beet


### Grouping Related Code in Modules

Modules let us organize code within a crate for readability and easy reuse.  
Modules also allow us to control the privacy of items, because code within a  
module is private by default. Private items are internal implementation details  
not available for outside use. We can choose to make modules and the items  
within them public, which exposes them to allow external code to use and depend  
on them.

As an example, let’s write a library crate that provides the functionality of a  
restaurant. We’ll define the signatures of functions but leave their bodies  
empty to concentrate on the organization of the code, rather than the  
implementation of a restaurant.

In the restaurant industry, some parts of a restaurant are referred to as front  
of house and others as back of house. Front of house is where customers are;  
this encompasses where the hosts seat customers, servers take orders and  
payment, and bartenders make drinks. Back of house is where the chefs and cooks  
work in the kitchen, dishwashers clean up, and managers do administrative work.  

To structure our crate in this way, we can organize its functions into nested  
modules. Create a new library named restaurant by running  
`cargo new restaurant --lib;` then enter the code in Listing 7-1 into src/lib.rs  
to define some modules and function signatures. Here’s the front of house  
section:

> Filename: src/lib.rs

```rust
mod front_of_house {
    mod hosting {
        fn add_to_waitlist() {}

        fn seat_at_table() {}
    }

    mod serving {
        fn take_order() {}

        fn serve_order() {}

        fn take_payment() {}
    }
}
```
> Listing 7-1: A front_of_house module containing other modules that then  
contain functions

We define a module with the `mod` keyword followed by the name of the module  
(in this case, front_of_house). The body of the module then goes inside curly  
brackets. Inside modules, we can place other modules, as in this case with the  
modules hosting and serving. Modules can also hold definitions for other items,  
such as : 
- structs, 
- enums, 
- constants, 
- traits, 
- functions.

> By using modules, we can group related definitions together and name why they’re  
related. Programmers using this code can navigate the code based on the groups  
rather than having to read through all the definitions, making it easier to find  
the definitions relevant to them. Programmers adding new functionality to this  
code would know where to place the code to keep the program organized.

Earlier, we mentioned that `src/main.rs` and `src/lib.rs` are called crate  
roots. The reason for their name is that the contents of either of these two  
files form a module named crate at the root of the crate’s module structure,  
known as the module tree.

> Listing 7-2 shows the module tree for the structure in Listing 7-1.

```sh
crate
 └── front_of_house
     ├── hosting
     │   ├── add_to_waitlist
     │   └── seat_at_table
     └── serving
         ├── take_order
         ├── serve_order
         └── take_payment
```

> Listing 7-2: The module tree for the code in Listing 7-1

This tree shows how some of the modules nest inside one another; for example,  
`hosting` nests inside `front_of_house.` The tree also shows that some modules are  
siblings to each other, meaning they’re defined in the same module; `hosting` and  
`serving` are siblings defined within `front_of_house`. If module A is contained  
inside module B, we say that module A is the child of module B and that module  
B is the parent of module A. Notice that the entire module tree is rooted under  
the implicit module named `crate`.

> The module tree might remind you of the `filesystem’s directory tree` on your  
computer; this is a very `apt comparison`! Just like directories in a filesystem,  
you use modules to organize your code. And just like files in a directory, we  
need a way to find our modules.

Paths for Referring to an Item in the Module Tree
---

To show Rust where to find an item in a module tree, we use a path in the same  
way we use a path when navigating a filesystem. To call a function, we need to  
know its path.

A path can take two forms:

- An **`absolute path`** is the full path starting from a crate root; for code  
from an external crate, the absolute path begins with the crate name, and for  
code from the current crate, it starts with the literal crate.  
- A **`relative path`** starts from the current module and uses self, super, or  
an identifier in the current module.  

Both absolute and relative paths are followed by one or more identifiers  
separated by double colons (::).

Returning to Listing 7-1, say we want to call the `add_to_waitlist` function. This  
is the same as asking:  

- what’s the path of the `add_to_waitlist` function?  

Listing 7-3 contains Listing 7-1 with some of the modules and functions  
removed.

> Filename: src/lib.rs

```rust
mod front_of_house {
    mod hosting {
        fn add_to_waitlist() {}
    }
}

pub fn eat_at_restaurant() {
    // Absolute path
    // We can use the **`crate`** keyword to start an absolute path because
    // - add_to_waitlist function is defined in the same crate as eat_at_restaurant
    crate::front_of_house::hosting::add_to_waitlist();

    // Relative path
    // Starting with a module name means that the path is relative
    front_of_house::hosting::add_to_waitlist();
}

/*
BY DEFAULT : @note Our preference in general is to specify absolute paths because it’s  
more likely we’ll want to move code definitions and item calls independently of  
each other. 
*/

```

> Listing 7-3: Calling the add_to_waitlist function using absolute and relative  
paths

We’ll show two ways to call the `add_to_waitlist` function from a new function  
`eat_at_restaurant` defined in the crate root. These paths are correct, but  
there’s another problem remaining that will prevent this example from  
compiling as-is. We’ll explain why in a bit.  


The first time we call the add_to_waitlist function in eat_at_restaurant, we use  
an absolute path. The add_to_waitlist function is defined in the same crate as  
eat_at_restaurant, which means we can use the `crate` keyword to start an  
absolute path. We then include each of the successive modules until we make our  
way to add_to_waitlist. You can imagine a filesystem with the same structure:  
we’d specify the path /front_of_house/hosting/add_to_waitlist to run the  
add_to_waitlist program; using the crate name to start from the crate root is  
like using / to start from the filesystem root in your shell.

The second time we call add_to_waitlist in eat_at_restaurant, we use a relative  
path. The path starts with front_of_house, the name of the module defined at the  
same level of the module tree as eat_at_restaurant. Here the filesystem  
equivalent would be using the path front_of_house/hosting/add_to_waitlist.  
Starting with a module name means that the path is relative.

Choosing whether to use a relative or absolute path is a decision you’ll make  
based on your project, and depends on whether you’re more likely to move item  
definition code separately from or together with the code that uses the item.  
For example :  
- If we move the `front_of_house` module AND the `eat_at_restaurant` function into  
a module named `customer_experience`, we’d need to update the absolute path to  
`add_to_waitlist`, but the `relative path would still be valid`. 
- However, if we moved the `eat_at_restaurant` function separately into a module  
named `dining`, the `absolute` path to the `add_to_waitlist` call would stay the same,  
but the relative path would need to be updated.  

> BY DEFAULT : @note Our preference in general is to specify absolute paths because it’s  
more likely we’ll want to move code definitions and item calls independently of  
each other.

In [14]:
mod front_of_house {
    mod hosting {
        fn add_to_waitlist() {}
    }
}

pub fn eat_at_restaurant() {
    // Absolute path
    // We can use the **`crate`** keyword to start an absolute path because
    // - add_to_waitlist function is defined in the same crate as eat_at_restaurant
    crate::front_of_house::hosting::add_to_waitlist();

    // Relative path
    // Starting with a module name means that the path is relative
    front_of_house::hosting::add_to_waitlist();
}

/*
BY DEFAULT : @note Our preference in general is to specify absolute paths because it’s  
more likely we’ll want to move code definitions and item calls independently of  
each other. 
*/

Error: module `hosting` is private

Error: module `hosting` is private

Let’s try to compile Listing 7-3 and find out why it won’t compile yet! The  
error we get is shown above ^

The error messages say that module hosting is private. In other words, we have  
the correct paths for the hosting module and the add_to_waitlist function, but  
Rust won’t let us use them because it doesn’t have access to the private  
sections. In Rust, all items (functions, methods, structs, enums, modules, and  
constants) are private to parent modules by default. If you want to make an item  
like a function or struct private, you put it in a module.

Items in a parent module can’t use the private items inside child modules, but  
items in child modules can use the items in their ancestor modules. This is  
because child modules wrap and hide their implementation details, but the child  
modules can see the context in which they’re defined. To continue with our  
metaphor, think of the privacy rules as being like the back office of a  
restaurant: what goes on in there is private to restaurant customers, but office  
managers can see and do everything in the restaurant they operate.

Rust chose to have the module system function this way so that hiding inner  
implementation details is the default. That way, you know which parts of the  
inner code you can change without breaking outer code. However, Rust does give  
you the option to expose inner parts of child modules’ code to outer ancestor  
modules by using the **`pub`** keyword to make an item public.